In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar3/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 50
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.4.1s"
merge_list = ["t2.4.4", "t2.4.2"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.1s.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

217922

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t2.4.1s+2.4.4+2.4.2


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.4.pkl
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.2.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

217922

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 217922
topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

3.2817845344543457

In [18]:
word_doc_matrix

<217922x740 sparse matrix of type '<class 'numpy.float64'>'
	with 6588637 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 400 edges.
It has only fully disconnected 11 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 40min 25s, sys: 1min 53s, total: 42min 19s
Wall time: 9min 45s


In [21]:
print(runtime_umap)

585.3469302654266


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 24min 22s, sys: 3.89 s, total: 24min 26s
Wall time: 22min 42s


In [23]:
print(runtime_umap_transform)

1362.173959493637


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_embedding_t2.4.1s+2.4.4+2.4.2.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_clusterable_embedding_t2.4.1s+2.4.4+2.4.2.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
4,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs50,50,250,6.239150,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
4,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs50,50,250,6.239150,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs450,450,50,3.676767,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
73,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs450,450,100,4.321940,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
74,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs450,450,150,4.868667,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
75,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs450,450,200,5.599749,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
4,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs50,50,250,6.239150,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs450,450,100,4.321940,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
74,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs450,450,150,4.868667,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
75,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs450,450,200,5.599749,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
76,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs450,450,250,6.240539,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
4,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs50,50,250,6.239150,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs450,450,150,4.868667,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
75,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs450,450,200,5.599749,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
76,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs450,450,250,6.240539,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
77,hdb,t2.4.1s+2.4.4+2.4.2,1,ss300cs450,450,300,6.870065,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
4,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs50,50,250,6.239150,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs450,450,200,5.599749,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
76,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs450,450,250,6.240539,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
77,hdb,t2.4.1s+2.4.4+2.4.2,1,ss300cs450,450,300,6.870065,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
78,hdb,t2.4.1s+2.4.4+2.4.2,1,ss350cs450,450,350,7.465997,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.4+2.4.2,1,ss50cs50,50,50,4.717498,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
1,hdb,t2.4.1s+2.4.4+2.4.2,1,ss100cs50,50,100,4.330479,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
2,hdb,t2.4.1s+2.4.4+2.4.2,1,ss150cs50,50,150,4.765043,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
3,hdb,t2.4.1s+2.4.4+2.4.2,1,ss200cs50,50,200,5.473252,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
4,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs50,50,250,6.239150,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.4.1s+2.4.4+2.4.2,1,ss250cs450,450,250,6.240539,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
77,hdb,t2.4.1s+2.4.4+2.4.2,1,ss300cs450,450,300,6.870065,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
78,hdb,t2.4.1s+2.4.4+2.4.2,1,ss350cs450,450,350,7.465997,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."
79,hdb,t2.4.1s+2.4.4+2.4.2,1,ss400cs450,450,400,8.554969,"rt_vect: 3.2817845344543457, min_df: 4, max_df...","hellinger, rt_umap_tf: 1362.173959493637, rt_u..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,technopagan/mss-saliency,-1,-1,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
1,snmnmin12/selective-search,-1,-1,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
2,google/chaps-linux,-1,-1,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
3,lukasolson/speech-router,211,87,72,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
4,XavierTecBr/qbr-admin,-1,-1,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217917,bhagatabhijeet/bhagatabhijeet,-1,-1,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
217918,LandGrey/SpringBootVulExploit,-1,-1,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
217919,mucomplex/Pentest_checklist,142,47,37,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1
217920,bigsen/Vision_Track,-1,18,-1,6,5,4,3,0,1,...,1,2,0,0,1,1,0,1,0,1


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_labels_results_t2.4.1s+2.4.4+2.4.2.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
4,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,22,11,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.492255,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


...

,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
4,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,22,11,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.492255,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
5,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,25,11,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.766478,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
6,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,28,11,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.952356,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
7,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,31,11,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.325231,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
8,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,34,11,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.700076,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
9,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,37,11,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",4.142833,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
4,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,22,11,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.492255,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
5,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,25,11,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.766478,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
6,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,28,11,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.952356,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
7,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,31,11,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.325231,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
8,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,34,11,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.700076,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
9,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,37,11,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",4.142833,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
4,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,22,11,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.492255,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
5,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,25,11,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.766478,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
6,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,28,11,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.952356,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
7,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,31,11,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.325231,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
8,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,34,11,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.700076,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
9,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,37,11,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",4.142833,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
4,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,22,11,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.492255,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
5,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,25,11,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.766478,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
6,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,28,11,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.952356,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
7,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,31,11,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.325231,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
8,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,34,11,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.700076,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
9,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,37,11,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",4.142833,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,10,11,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.281459,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
1,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,13,11,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.415602,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
2,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,16,11,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.666748,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
3,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,19,11,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",1.987270,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
4,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,22,11,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.492255,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
5,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,25,11,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.766478,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
6,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,28,11,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",2.952356,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
7,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,31,11,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.325231,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
8,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,34,11,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",3.700076,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."
9,kmeans-umapt,t2.4.1s+2.4.4+2.4.2,1,37,11,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",4.142833,"(hellinger, rt_umap_tf: 1362.173959493637, rt_..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,technopagan/mss-saliency,1,0,10,1,17,1,2,27,26,22,5,15,1,12
1,snmnmin12/selective-search,1,0,1,7,17,1,2,27,26,22,38,36,11,1
2,google/chaps-linux,7,5,2,4,0,0,21,19,30,11,30,31,44,2
3,lukasolson/speech-router,6,9,9,2,4,2,1,15,16,2,3,25,14,6
4,XavierTecBr/qbr-admin,2,10,12,11,16,24,25,20,33,14,9,18,10,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217917,bhagatabhijeet/bhagatabhijeet,0,10,12,11,16,24,25,20,33,14,22,18,37,21
217918,LandGrey/SpringBootVulExploit,4,1,0,18,18,3,6,22,32,17,11,38,16,16
217919,mucomplex/Pentest_checklist,7,5,5,0,10,7,9,14,22,12,39,3,20,33
217920,bigsen/Vision_Track,0,2,3,10,3,6,19,9,24,15,15,6,5,4


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_labels_results_t2.4.1s+2.4.4+2.4.2.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405
4,kmeans,t2.4.1s+2.4.4+2.4.2,1,22,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",38.219313


...

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405
4,kmeans,t2.4.1s+2.4.4+2.4.2,1,22,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",38.219313
5,kmeans,t2.4.1s+2.4.4+2.4.2,1,25,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",50.285182
6,kmeans,t2.4.1s+2.4.4+2.4.2,1,28,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",47.177012
7,kmeans,t2.4.1s+2.4.4+2.4.2,1,31,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",55.124839
8,kmeans,t2.4.1s+2.4.4+2.4.2,1,34,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",53.879157
9,kmeans,t2.4.1s+2.4.4+2.4.2,1,37,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",65.632617


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405
4,kmeans,t2.4.1s+2.4.4+2.4.2,1,22,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",38.219313
5,kmeans,t2.4.1s+2.4.4+2.4.2,1,25,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",50.285182
6,kmeans,t2.4.1s+2.4.4+2.4.2,1,28,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",47.177012
7,kmeans,t2.4.1s+2.4.4+2.4.2,1,31,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",55.124839
8,kmeans,t2.4.1s+2.4.4+2.4.2,1,34,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",53.879157
9,kmeans,t2.4.1s+2.4.4+2.4.2,1,37,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",65.632617


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405
4,kmeans,t2.4.1s+2.4.4+2.4.2,1,22,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",38.219313
5,kmeans,t2.4.1s+2.4.4+2.4.2,1,25,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",50.285182
6,kmeans,t2.4.1s+2.4.4+2.4.2,1,28,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",47.177012
7,kmeans,t2.4.1s+2.4.4+2.4.2,1,31,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",55.124839
8,kmeans,t2.4.1s+2.4.4+2.4.2,1,34,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",53.879157
9,kmeans,t2.4.1s+2.4.4+2.4.2,1,37,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",65.632617


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405
4,kmeans,t2.4.1s+2.4.4+2.4.2,1,22,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",38.219313
5,kmeans,t2.4.1s+2.4.4+2.4.2,1,25,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",50.285182
6,kmeans,t2.4.1s+2.4.4+2.4.2,1,28,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",47.177012
7,kmeans,t2.4.1s+2.4.4+2.4.2,1,31,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",55.124839
8,kmeans,t2.4.1s+2.4.4+2.4.2,1,34,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",53.879157
9,kmeans,t2.4.1s+2.4.4+2.4.2,1,37,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",65.632617


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.4+2.4.2,1,10,10,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",13.085954
1,kmeans,t2.4.1s+2.4.4+2.4.2,1,13,13,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",17.510185
2,kmeans,t2.4.1s+2.4.4+2.4.2,1,16,16,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",21.814248
3,kmeans,t2.4.1s+2.4.4+2.4.2,1,19,19,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",27.292405
4,kmeans,t2.4.1s+2.4.4+2.4.2,1,22,22,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",38.219313
5,kmeans,t2.4.1s+2.4.4+2.4.2,1,25,25,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",50.285182
6,kmeans,t2.4.1s+2.4.4+2.4.2,1,28,28,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",47.177012
7,kmeans,t2.4.1s+2.4.4+2.4.2,1,31,31,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",55.124839
8,kmeans,t2.4.1s+2.4.4+2.4.2,1,34,34,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",53.879157
9,kmeans,t2.4.1s+2.4.4+2.4.2,1,37,37,"rt_vect: 3.2817845344543457, min_df: 4, max_df...",65.632617


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,technopagan/mss-saliency,4,0,0,18,19,21,7,14,10,5,0,36,2,32
1,snmnmin12/selective-search,4,0,0,18,19,21,7,14,10,1,15,36,27,32
2,google/chaps-linux,9,6,12,3,16,16,21,4,22,35,38,30,31,34
3,lukasolson/speech-router,1,3,15,5,3,9,13,24,32,31,20,23,36,35
4,XavierTecBr/qbr-admin,2,12,3,15,20,0,0,3,26,10,22,11,1,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217917,bhagatabhijeet/bhagatabhijeet,6,7,13,16,8,8,27,17,15,2,17,18,4,4
217918,LandGrey/SpringBootVulExploit,2,12,3,15,20,24,9,30,5,10,22,13,10,3
217919,mucomplex/Pentest_checklist,9,11,8,1,13,17,5,20,20,28,32,10,0,9
217920,bigsen/Vision_Track,7,5,0,9,19,1,1,14,10,6,15,37,6,27


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_labels_results_t2.4.1s+2.4.4+2.4.2.pkl
